Velocity controller design
===========================

In this task we will demonstrate how to design and implement a velocity PI-controller for the following model: 

$\dot{v} = \frac{F_x}{m} - \frac{c_d}{m}v^2$

The system input $F_x$ can be split into the control input $u$ and external disturbance forces $d$. We can therefore rewrite the equation 

$\dot{v} = \frac{u + d}{m} - \frac{c_d}{m}v^2$

and analyze its behavior with respect to external input signals d. The control law is built from a proportional $K_p$ and an integral feedback part $K_i$: 

$u = K_pe + K_i\int{e \text{dt}}$

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

ratio = [16, 10]
linewidth = 4
plt.rcParams["figure.figsize"] = ratio
plt.rcParams.update({"font.size": 22})

# step size in seconds
tS = 0.01
# number of steps
N = 1000
# the resulting length of the solution equals tS*(N-1) seconds
t = np.arange(0, tS * N, tS)
# vehicle mass
mass = 1200
# lumped drag coefficient
cd = 0.45

### Even though PI-controllers are often tuned empirically, it is quite helpful to determine an initial set of parameters using standard methods for control system design. We will go through these steps now and derive a suitable set of controller parameters: 

## First Step: Linearization

The system itself is nonlinear due to the term $v^2$. We therefore have to linearize around an operating point to apply linear control system design methods. We label the state $x = v$ and the input $F_x$ is splitted into the control input $u$ and the disturbance input $d$. The system equation in its general form can now be written as: 

$\dot{x} = f(x, u, d) = \frac{u + d}{m} - \frac{c_d}{m}v^2$

Linearization with respect to the state: 

$\frac{\partial f(x, u, d)}{\partial x} = -\frac{2c_d}{m}v_0$

Linearization with respect to the input: 

$\frac{\partial f(x, u, d)}{\partial u} = \frac{1}{m}$

The linearized system equation around the operating point $v_0$ is now: 

$\Delta \dot{v} = \frac{u}{m} + \frac{d}{m} -\frac{2c_d}{m}v_0\Delta v$

## Second Step: Controller Design

To design the PI controller, we calculate the transfer function with the laplace variable $s$ from $u$ to $\Delta v$. We can neglect the disturbance input here as the linear system theory allows us to treat the inputs independently (superposition principle). 

$\Delta vs = \frac{u(s)}{m} -\frac{2c_d}{m}v_0\Delta v(s)$

By reformulation we get

$G_p(s) = \frac{v(s)}{u(s)} = \frac{1}{\left(ms+2c_dv_0\right)}$

Using the PI controller transfer function from the control error $e$ to the control value $u$

$G_r(s) \frac{u(s)}{e(s)} = \frac{K_ps+K_i}{s}$

we get the closed-loop transfer function

$G_c(s) = \frac{G_p(s)G_r(s)}{1+G_p(s)G_r(s)} = \frac{K_ps+K_i}{K_ps+K_i+s\left(ms+2c_dv_0\right)} = \frac{1}{m}\frac{K_ps+K_i}{s^2 + \frac{K_p + 2c_dv_0}{m}s + \frac{K_i}{m}}$

We will obtain the gains $K_p$ and $K_i$ via coefficient comparison. The target dynamics for our closed-loop are specified as a pole at -3+3j and -3-3j. The resulting target polynomial is 

$\left(s+3+3j\right)\left(s+3-3j\right) = s^2 + \frac{K_p + 2c_dv_0}{m}s + \frac{K_i}{m}$

$s^2 + 6s + 18 = s^2 + \frac{K_p + 2c_dv_0}{m}s + \frac{K_i}{m}$

The controller gains are therefore calculated to be

$K_p = 6m - 2c_dv_0$ for a operating point of $v_0 = 10$mps

$K_i = 18m$

In [ ]:
# implementation of the PI control law
v0 = 10
Kp = 6 * mass - 2 * cd * v0
Ki = 18 * mass
print("The proportional gain is: " + str(Kp))
print("The integral gain is: " + str(Ki))


def control(e, old_integrator_value, tS, Kp=Kp, Ki=Ki):
    # apply numerical integration to obtain the next value of the integral
    integrator_value = old_integrator_value + tS * e
    # calculate control law
    u = Kp * e + Ki * integrator_value
    return u, integrator_value

In [ ]:
# initialize the solution with all zeros
v = np.zeros(N)
# specify the initial condition with a certain speed
v[0] = v0
# specify the disturbance input (0N for the first 2.5 seconds and -3000 for the rest of the horizon)
d = np.hstack((np.full(int(N / 4), 0), np.full(int(N * 3 / 4), -3000)))
plt.plot(t, d, linewidth=linewidth)
plt.grid()
plt.xlabel("Time in s")
plt.ylabel("Disturbance force in N")
plt.show()

In [ ]:
# we will now determine the solution to the controlled differential equation specified above
# store the control inputs
u_log = np.zeros(N)
# initialize the integrator with zero
old_integrator_value = 0
# define a velocity setpoint
v_set = 10
for i in range(1, v.shape[0]):
    # determine control error
    e = v_set - v[i - 1]
    # determine control value
    u, old_integrator_value = control(e, old_integrator_value, tS)
    u_log[i] = u
    # apply euler forward integration (x(k+1) = x(k) + tS*d_x(k))
    v[i] = v[i - 1] + tS * (u / mass + d[i - 1] / mass - cd / mass * v[i - 1] ** 2)

# visualize solution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(32, 10))
ax1.plot(t, v, label="Actual value", linewidth=linewidth)
ax1.plot(t, np.full_like(v, v_set), label="Target value", linewidth=linewidth)
ax1.grid()
ax1.set_xlabel("Time in s")
ax1.set_ylabel("Velocity in mps")
ax1.legend()
ax2.plot(t, u_log, linewidth=linewidth)
ax2.grid()
ax2.set_xlabel("Time in s")
ax2.set_ylabel("Control input in N")
plt.show()